1. GRU

In [1]:
with open("./donquixote.txt", "r", encoding="utf8") as f:
    text = f.read().lower()

print(len(text))

2298195


In [2]:
text[:500]

'don quixote\n\n\n\nby miguel de cervantes\n\n\n\ntranslated by john ormsby\n\ncontents volume i\n\n\nintroduction\n\nprefaratory\n\ncervantes\n\n‘don quixote’\n\nthe author’s preface\n\ncommendatory verses\n\n\n\n\nchapter i which treats of the character and\npursuits of the famous gentleman don quixote of la mancha \n\nchapter ii which treats of the first sally the ingenious\ndon quixote made from home \n\nchapter iii\nwherein is related the droll way in which don quixote had himself dubbed a\nknight \n\nchapter iv of what happened'

In [3]:
import numpy as np
from tqdm import tqdm

maxlen = 60
step = 3
sequences = []
next_chars = []

for i in tqdm(range(0, len(text) - maxlen, step)):
    sequences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
len(sequences)

100%|██████████| 766045/766045 [00:00<00:00, 1379628.13it/s]


766045

In [4]:
for i in range(5): print(repr(sequences[i])," : ", next_chars[i])

'don quixote\n\n\n\nby miguel de cervantes\n\n\n\ntranslated by john '  :  o
' quixote\n\n\n\nby miguel de cervantes\n\n\n\ntranslated by john orm'  :  s
'ixote\n\n\n\nby miguel de cervantes\n\n\n\ntranslated by john ormsby'  :  

'te\n\n\n\nby miguel de cervantes\n\n\n\ntranslated by john ormsby\n\nc'  :  o
'\n\n\nby miguel de cervantes\n\n\n\ntranslated by john ormsby\n\ncont'  :  e


In [5]:
chars = sorted(list(set(text)))
print(chars)

['\n', ' ', '!', '&', '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '>', '?', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'á', 'æ', 'è', 'é', 'ë', 'í', 'ñ', 'ó', 'ù', 'ú', 'œ', '—', '‘', '’', '“', '”']


In [6]:
char_indices = dict((char, chars.index(char)) for char in chars)
print(char_indices)

{'\n': 0, ' ': 1, '!': 2, '&': 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, '/': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '>': 22, '?': 23, '_': 24, 'a': 25, 'b': 26, 'c': 27, 'd': 28, 'e': 29, 'f': 30, 'g': 31, 'h': 32, 'i': 33, 'j': 34, 'k': 35, 'l': 36, 'm': 37, 'n': 38, 'o': 39, 'p': 40, 'q': 41, 'r': 42, 's': 43, 't': 44, 'u': 45, 'v': 46, 'w': 47, 'x': 48, 'y': 49, 'z': 50, 'à': 51, 'á': 52, 'æ': 53, 'è': 54, 'é': 55, 'ë': 56, 'í': 57, 'ñ': 58, 'ó': 59, 'ù': 60, 'ú': 61, 'œ': 62, '—': 63, '‘': 64, '’': 65, '“': 66, '”': 67}


In [7]:
indices_char = dict((chars.index(char), char) for char in chars)
print(indices_char)

{0: '\n', 1: ' ', 2: '!', 3: '&', 4: '(', 5: ')', 6: ',', 7: '-', 8: '.', 9: '/', 10: '0', 11: '1', 12: '2', 13: '3', 14: '4', 15: '5', 16: '6', 17: '7', 18: '8', 19: '9', 20: ':', 21: ';', 22: '>', 23: '?', 24: '_', 25: 'a', 26: 'b', 27: 'c', 28: 'd', 29: 'e', 30: 'f', 31: 'g', 32: 'h', 33: 'i', 34: 'j', 35: 'k', 36: 'l', 37: 'm', 38: 'n', 39: 'o', 40: 'p', 41: 'q', 42: 'r', 43: 's', 44: 't', 45: 'u', 46: 'v', 47: 'w', 48: 'x', 49: 'y', 50: 'z', 51: 'à', 52: 'á', 53: 'æ', 54: 'è', 55: 'é', 56: 'ë', 57: 'í', 58: 'ñ', 59: 'ó', 60: 'ù', 61: 'ú', 62: 'œ', 63: '—', 64: '‘', 65: '’', 66: '“', 67: '”'}


In [8]:
X = []
y = []
for i, seq in tqdm(enumerate(sequences)):
    x = [char_indices[char] for char in seq]
    X.append(x)
    y.append([char_indices[next_chars[i]]])

X = np.array(X)
y = np.array(y)
X.shape, y.shape

766045it [00:11, 64052.09it/s] 


((766045, 60), (766045, 1))

In [9]:
#!pip install tensorflow

In [10]:
import tensorflow as tf

emb_size = 8
num_chars = len(chars)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(num_chars, emb_size, input_length=maxlen),
    tf.keras.layers.Conv1D(32, 5, activation='relu'),    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=num_chars, activation='softmax')
]) 

model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 8)             544       
                                                                 
 conv1d (Conv1D)             (None, 56, 32)            1312      
                                                                 
 flatten (Flatten)           (None, 1792)              0         
                                                                 
 dropout (Dropout)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 68)                121924    
                                                                 
Total params: 123,780
Trainable params: 123,780
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# with tf.device("/CPU:0"):
model.fit(X, y, epochs=1, batch_size=32)

10395/23939 [============>.................] - ETA: 53s - loss: 2.3872 - accuracy: 0.3205

In [ ]:
import random

start_index = random.randint(0, len(text) - maxlen - 1)
base_text = text[start_index: start_index + maxlen]
base_text

In [ ]:
import sys

next_char = 100
new_text = base_text
sys.stdout.write(new_text)

for _ in range(next_char):
    x = np.array([[char_indices[char] for char in base_text]])  
    predicted = np.argmax(model.predict(np.array([[char_indices[char] for char in new_text]]), verbose=0), axis=-1)
    new_char = indices_char[predicted[0]]
    new_text = new_text[1:] + new_char
    sys.stdout.write(new_char)

2. Konvulúciós

In [ ]:
with open("./donquixote.txt", "r", encoding="utf8") as f:
    text = f.read().lower()

print(len(text))

In [ ]:
text[:500]

In [ ]:
import numpy as np
from tqdm import tqdm

maxlen = 60
step = 3
sentences = []
next_chars = []

for i in tqdm(range(0, len(text) - maxlen, step)):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
len(sentences)

In [ ]:
for i in range(5): print(repr(sentences[i])," : ", next_chars[i])

In [ ]:
chars = sorted(list(set(text)))
print(chars)

In [ ]:
char_indices = dict((char, chars.index(char)) for char in chars)
print(char_indices)

In [ ]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in tqdm(enumerate(sentences)):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print("x", x.shape)
print("y", y.shape)

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.GRU(16, input_shape=(maxlen, len(chars))),
    tf.keras.layers.Dense(units=len(chars), activation='softmax')
]) 

model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

model.summary()

In [ ]:
with tf.device("/CPU:0"):
    model.fit(x, 
            y,
            batch_size=128,
            epochs=1)

In [ ]:
import random

start_index = random.randint(0, len(text) - maxlen - 1)
base_text = text[start_index: start_index + maxlen]
base_text

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import sys
temperatures = [0.2, 0.5, 1.0, 1.2]
gen_characters = 200

for temp in temperatures:
    print("Temp: ", temp)
    generated_text = base_text
    print(generated_text)
    for i in range(gen_characters):
        sampled = np.zeros((1, maxlen, len(chars)))    
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.          
        
        preds = model.predict(sampled, verbose=0)[0]      
        
        next_index = sample(preds, temp)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()

    print()

3. LSTM

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np 

In [ ]:
with open("./donquixote.txt", "r", encoding="utf8") as f:
    text = f.read().lower()

print(len(text))

In [ ]:
lines = text.split("\n")

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
num_words = len(tokenizer.word_index) + 1

In [ ]:
print(lines[3])
tokenizer.texts_to_sequences([lines[3]])[0]

In [ ]:
input_sequences = []

for line in lines:
    tokens = tokenizer.texts_to_sequences([line])[0]

    for i in range(1, len(tokens)):
        input_sequences.append(tokens[:i+1])

In [ ]:
input_sequences[0]

In [ ]:
max_sequence_len = max([len(i) for i in input_sequences])

In [ ]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
np.random.shuffle(input_sequences)
X, y = input_sequences[:,:-1], np.expand_dims(input_sequences[:,-1], axis=1)

In [ ]:
X.shape

In [ ]:
X[0]

In [ ]:
type(X)

In [ ]:
y.shape

In [ ]:
y

In [ ]:
num_words

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model

emb_size = 256

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(num_words, emb_size, input_length=max_sequence_len - 1),
    tf.keras.layers.LSTM(120),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dense(units=num_words, activation='softmax')
]) 

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer="adam", metrics=['accuracy'])

model.summary()

In [ ]:
plot_model(model, show_shapes=True)

In [ ]:
callback = EarlyStopping(monitor="loss", patience=3)
history = model.fit(X, y, epochs=3, batch_size=128, verbose=1, callbacks=[callback])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='(training data)',color='blue')
plt.title('Neural Network training loss')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='(training data)',color='blue')
plt.title('Neural Network training accuracy')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
import sys

text = "The philosophy of the dogmatists, it is to be hoped, was only a promise for thousands of years afterwards"
next_words = 100

print(text)
for _ in range(next_words):
    tokens = tokenizer.texts_to_sequences([text])[0]
    tokens = pad_sequences([tokens], maxlen=max_sequence_len-1, padding='pre')   
    predicted = np.argmax(model.predict(tokens, verbose=0), axis=-1)
 
    output_word = ""

    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break

    sys.stdout.write(output_word + " ")
    text += " " + output_word